# Información adicional TFM: Aprendizaje profundo por refuerzo en el entorno Google Football

* Alumno: Ángel Murcia Díaz

* Director: Juan Gómez Romero

> En este trabajo se ha realizado una investigación acerca de una rama de la *Inteligencia artificial*, el *Aprendizaje por Refuerzo, tanto de forma teórica como práctica. El objetivo que persigue el proyecto es doble: por un parte, estudiar los fundamentos teóricos de los algoritmos de *Aprendizaje por Refuerzo Profundo*, un tipo de *Aprendizaje por Refuerzo* que utiliza *Redes Neuronales*; por otra, comprobar si es posible aplicar estos algoritmos para realizar *Aprendizaje Progresivo* (*curriculum learning*, en inglés) en problemas que se organizan en niveles de dificultad creciente. 

> Para ello, en primer lugar se ha llevado a cabo una extensa revisión del marco teórico del *Aprendizaje por Refuerzo Profundo*, estudiando los conceptos clave, los algoritmos propuestos en los últimos años y las implementaciones públicas disponibles. En segundo lugar, se han implementado y ejecutado una serie de experimentos utilizando el entorno virtual *Google Research Football*, que simula un partido de fútbol y permite controlar a los jugadores como si se tratara de un videojuego.  

> Las conclusiones más importantes del trabajo son: (1) que los algoritmos de *Aprendizaje por Refuerzo Profundo* son muy útiles incluso en problemas con espacios de estados y acciones complejos, (2) que el *Aprendizaje Progresivo* es más rápido y efectivo que el aprendizaje desde cero, incluso cuando las tareas más complejas no contienen completamente a las más simples. 

* El presente cuaderno Python se utiliza para la realización del código de este TFM.

* Se ha escogido *Google Colab* sobretodo por la oportunidad que brinda al usuario a utilizar la GPU de Google, la cuál viene estupenda para la resolución de este tipo de problemas, puesto que se utilizarán entre otras alternativas *Redes Neuronales Convolucionales* (CNN).

# Información adiccional del cuaderno Proceso_Experimental_Avanzado



En este cuaderno se procede a la realización del proceso exprimental avanzado, que consiste en la aplicación del aprendizaje progresivo ("curriculum learning"), es decir, un tipo de aprendizaje que comienza en escenarios más sencillos para acabar en escenarios más complejos del entorno.

Destacar que se han planteado 3 conjuntos de escenarios de dificultad progresiva, es decir, recorridos para entrenar el agente de forma progresiva, estos son:

* Conjunto 1 de escenarios de dificultad progresiva (partido): recorrido en escenarios de partidos de 11 contra 11.
* Conjunto 2 de escenarios de dificultad progresiva (contragolpe): recorrido en escenarios de contragolpe.
* Conjunto 3 de escenarios de dificultad progresiva (completo): recorrido completo, es decir, este abarca todas las situaciones del juego desde las más simples hasta las más complejas (partido de 11 contra 11 en dificultad difícil).

Se utiliza PPO2 como algoritmo y CNN e IMPALA como red subyacente, puesto que son las configuraciones que mejores resultados obtienen en la experimentación básica

# Instalación de las librerías y paquetes necesarios

In [ ]:
!python3 -m pip install --upgrade pip setuptools

In [ ]:
!pip3 install tensorflow-gpu==1.15.*

In [ ]:
!pip3 install dm-sonnet==1.*

In [ ]:
!pip3 install git+https://github.com/openai/baselines.git@master

In [ ]:
!apt-get update
!apt-get install libsdl2-gfx-dev libsdl2-ttf-dev

!git clone -b v2.1 https://github.com/google-research/football.git
!mkdir -p football/third_party/gfootball_engine/lib

!wget https://storage.googleapis.com/gfootball/prebuilt_gameplayfootball_v2.1.so -O football/third_party/gfootball_engine/lib/prebuilt_gameplayfootball.so
!cd football && GFOOTBALL_USE_PREBUILT_SO=1 pip3 install .

#Conexión con Drive privado

* Para poder guardar los checkpoint y los monitor, de tal forma que si se cae la sesión de *Colab* no hay ningún problema, puesto que cuando se cae la sesión se pierden los archivos en *Colab*.

In [ ]:
from google.colab import drive

drive.mount("/content/gdrive")

# Celdas de código para bateria de experimentos del **Desarrollo Experimental Avanzado**


In [ ]:
# Fuente de código original: Google Football Research

#Import necesarios

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import multiprocessing
import os
from absl import app
from absl import flags
from baselines import logger
from baselines.bench import monitor
from baselines.common.vec_env.subproc_vec_env import SubprocVecEnv
from baselines.ppo2 import ppo2
from baselines.a2c import a2c
from baselines.deepq import deepq
import gfootball.env as football_env
from gfootball.examples import models  

from baselines.common import plot_util as pu

In [ ]:
#Funcion para la creación del entorno deseado de Google Football utilizada para la creacion posterior de vectores de entorno
def create_football_env(iprocess, ruta_completa, tipo_entorno):
  # Creación del entorno Google Football con el tipo de entorno a utilizar, tipo de recompensa, etc.
  env = football_env.create_environment(
      env_name=tipo_entorno, stacked=('stacked' in 'extracted_stacked'),
      rewards='scoring,checkpoints',
      write_goal_dumps=False and (iprocess == 0),
      write_full_episode_dumps=False and (iprocess == 0),
      render=False and (iprocess == 0),
      dump_frequency=50 if False and iprocess == 0 else 0)
  # Configurar Monitor para el seguimiento de cada entorno por separado
  env = monitor.Monitor(env, ruta_completa and os.path.join(ruta_completa, str(iprocess)))
  # Devolver entorno creado
  return env

#Funcion para el entrenamiento utilizando PPO2 de un agente en el un entorno de Google Football creado previamente
def PPO2_run_agent_train(_, ruta_completa, tipo_entorno, num_env, red, pasos, semilla, ruta_save_inicial):
  # Creacion del vector de entornos con entornos como se especifique
  vec_env = SubprocVecEnv([
      (lambda _i=i: create_football_env(_i, ruta_completa, tipo_entorno))
      for i in range(num_env)
  ], context=None)
  # Importación de Tensorflow version 1 compatible con librería Baselines
  import tensorflow.compat.v1 as tf
  # Configuración de variables de configuración del entorno
  ncpu = multiprocessing.cpu_count()
  config = tf.ConfigProto(allow_soft_placement=True,
                          intra_op_parallelism_threads=ncpu,
                          inter_op_parallelism_threads=ncpu)
  config.gpu_options.allow_growth = True
  tf.Session(config=config, graph=tf.Graph()).__enter__()
  # Entrenamiento del agente con los parametros especificados como red utilizada, número de pasos, etc.
  if ruta_save_inicial == None:
    model = ppo2.learn(network=red,
              total_timesteps=pasos,
              env=vec_env,
              seed=semilla
              )
  else:
    model = ppo2.learn(network=red,
          total_timesteps=pasos,
          env=vec_env,
          load_path=ruta_save_inicial,
          seed=semilla
          )
  return model

#Funcion para el entrenamiento utilizando A2C de un agente en el un entorno de Google Football creado previamente
def A2C_run_agent_train(_, ruta_completa, tipo_entorno, num_env, red, pasos, semilla, ruta_save_inicial):
  # Creacion del vector de entornos con entornos como se especifique
  vec_env = SubprocVecEnv([
      (lambda _i=i: create_football_env(_i, ruta_completa, tipo_entorno))
      for i in range(num_env)
  ], context=None)
  # Importación de Tensorflow version 1 compatible con librería Baselines
  import tensorflow.compat.v1 as tf
  # Configuración de variables de configuración del entorno
  ncpu = multiprocessing.cpu_count()
  config = tf.ConfigProto(allow_soft_placement=True,
                          intra_op_parallelism_threads=ncpu,
                          inter_op_parallelism_threads=ncpu)
  config.gpu_options.allow_growth = True
  tf.Session(config=config, graph=tf.Graph()).__enter__()
  # Entrenamiento del agente con los parametros especificados como red utilizada, número de pasos, etc.
  if ruta_save_inicial == None:
    model = a2c.learn(network=red,
              total_timesteps=pasos,
              env=vec_env,
              seed=semilla
              )
  else:
    model = a2c.learn(network=red,
              total_timesteps=pasos,
              env=vec_env,
              load_path=ruta_save_inicial,
              seed=semilla
              )
       
  return model

#Funcion para el entrenamiento utilizando DQN de un agente en el un entorno de Google Football creado previamente
def DQN_run_agent_train(_, ruta_completa, tipo_entorno, num_env, red, pasos, semilla, ruta_save_inicial):
  # Creacion de entorno simple
  env = football_env.create_environment(
    env_name=tipo_entorno,
    rewards='scoring,checkpoints')
  #Creacion de un entorno de forma simple (solo permite uno)
  env = monitor.Monitor(env, ruta_completa and os.path.join(ruta_completa))
  # Importación de Tensorflow version 1 compatible con librería Baselines
  import tensorflow.compat.v1 as tf
  # Configuración de variables de configuración del entorno
  ncpu = multiprocessing.cpu_count()
  config = tf.ConfigProto(allow_soft_placement=True,
                          intra_op_parallelism_threads=ncpu,
                          inter_op_parallelism_threads=ncpu)
  config.gpu_options.allow_growth = True
  tf.Session(config=config, graph=tf.Graph()).__enter__()
  # Entrenamiento del agente con los parametros especificados como red utilizada, número de pasos, etc.
  if ruta_save_inicial == None:
    model = deepq.learn(network=red,
              total_timesteps=pasos,
              env=env,
              seed=semilla
              )
  else:
    model = deepq.learn(network=red,
              total_timesteps=pasos,
              env=env,
              load_path=ruta_save_inicial,
              seed=semilla
              )
  return model

## Bateria de experimentos correspondientes con el conjunto 1 de escenarios de dificultad progresiva (partido)

In [ ]:
#Creación de vectores de parámetros para que sean recorridos y realizar todas las combinaciones posibles en la experimentación
#VEctor con diferentes escenarios del entorno
vector_tipos_env = ['academy_single_goal_versus_lazy', '11_vs_11_easy_stochastic', '11_vs_11_stochastic', '11_vs_11_hard_stochastic']
#Vector con numero de multi-entornos a utilizar
vector_numero_env = [8]
#Vector con tipo de red subyacente
vector_red =  ['gfootball_impala_cnn','cnn']
#Vector con numeros de timesteps a utilizar
vector_pasos = [1000000]
#Vector con semillas a utilizar
vector_seed = [0]

In [ ]:
# Ruta para el save inicial desde el que partir en el entrenamiento progresivo

#Ruta base para la experimentacion básica
ruta_base_experimentacion_avanzada= "/content/gdrive/My Drive/TFM/resultados/Experimentacion_Avanzada/"

# Inicializado a None puesto que en el primer entreno no hay save
save_inicial = None
for pasos in vector_pasos:
  for semilla in vector_seed:
    for red in vector_red:
      for num_env in vector_numero_env:
        for tipo in vector_tipos_env:
          # Ruta completa para PPO2
          ruta_completa_ppo2 = ruta_base_experimentacion_avanzada + "PARTIDO/" +  str(pasos) + "_timesteps" + "/" + str(semilla) + "_seed" + "/" + "PPO2" + "/" + red + "/" + str(num_env) + "/" + tipo
          logger.configure(dir=ruta_completa_ppo2)
          print(logger.get_dir())
          print(save_inicial)
          model = PPO2_run_agent_train(_, ruta_completa_ppo2, tipo, num_env, red, pasos, semilla, save_inicial)
          model.save(ruta_completa_ppo2 + "/checkpoints/final_save")
          save_inicial = ruta_completa_ppo2 + "/checkpoints/final_save"
          results = pu.load_results(ruta_completa_ppo2)
          grafica = pu.plot_results(results, average_group=True)
          grafica[0].savefig(ruta_completa_ppo2 + "/final_grafica.png")

## Bateria de experimentos correspondientes con el conjunto 2 de escenarios de dificultad progresiva (contragolpe)

In [ ]:
#Creación de vectores de parámetros para que sean recorridos y realizar todas las combinaciones posibles en la experimentación
#Vector con diferentes escenarios del entorno
vector_tipos_env = ['academy_counterattack_easy', 'academy_counterattack_hard']    
#Vector con numero de multi-entornos a utilizar
vector_numero_env = [8]
#Vector con tipo de red subyacente
vector_red =  ['gfootball_impala_cnn','cnn']
#Vector con numeros de timesteps a utilizar
vector_pasos = [1000000]
#Vector con semillas a utilizar
vector_seed = [0]

In [ ]:
# Ruta para el save inicial desde el que partir en el entrenamiento progresivo

#Ruta base para la experimentacion básica
ruta_base_experimentacion_avanzada= "/content/gdrive/My Drive/TFM/resultados/Experimentacion_Avanzada/"

# Inicializado a None puesto que en el primer entreno no hay save
save_inicial = None
for pasos in vector_pasos:
  for semilla in vector_seed:
    for red in vector_red:
      for num_env in vector_numero_env:
        for tipo in vector_tipos_env:
          # Ruta completa para PPO2
          ruta_completa_ppo2 = ruta_base_experimentacion_avanzada + "CONTRAGOLPE/" +  str(pasos) + "_timesteps" + "/" + str(semilla) + "_seed" + "/" + "PPO2" + "/" + red + "/" + str(num_env) + "/" + tipo
          logger.configure(dir=ruta_completa_ppo2)
          print(logger.get_dir())
          print(save_inicial)
          model = PPO2_run_agent_train(_, ruta_completa_ppo2, tipo, num_env, red, pasos, semilla, save_inicial)
          model.save(ruta_completa_ppo2 + "/checkpoints/final_save")
          save_inicial = ruta_completa_ppo2 + "/checkpoints/final_save"
          results = pu.load_results(ruta_completa_ppo2)
          grafica = pu.plot_results(results, average_group=True)
          grafica[0].savefig(ruta_completa_ppo2 + "/final_grafica.png")

## Bateria de experimentos correspondientes con el conjunto 3 de escenarios de dificultad progresiva (completo)

In [ ]:
#Creación de vectores de parámetros para que sean recorridos y realizar todas las combinaciones posibles en la experimentación
#Vector con diferentes escenarios del entorno
vector_tipos_env = ['academy_empty_goal_close', 'academy_empty_goal', 'academy_run_to_score', 'academy_run_to_score_with_keeper',
                    'academy_pass_and_shoot_with_keeper', 'academy_run_pass_and_shoot_with_keeper', '1_vs_1_easy', 
                    'academy_3_vs_1_with_keeper', 'academy_counterattack_easy' , 'academy_single_goal_versus_lazy', '5_vs_5',
                    '11_vs_11_easy_stochastic', 'academy_counterattack_hard', '11_vs_11_stochastic', '11_vs_11_hard_stochastic']
#Vector con numero de multi-entornos a utilizar
vector_numero_env = [8]
#Vector con tipo de red subyacente
vector_red =  ['gfootball_impala_cnn','cnn']
#Vector con numeros de timesteps a utilizar
vector_pasos = [1000000]
#Vector con semillas a utilizar
vector_seed = [0]

In [ ]:
# Ruta para el save inicial desde el que partir en el entrenamiento progresivo

#Ruta base para la experimentacion básica
ruta_base_experimentacion_avanzada= "/content/gdrive/My Drive/TFM/resultados/Experimentacion_Avanzada/"

# Inicializado a None puesto que en el primer entreno no hay save
save_inicial = None
for pasos in vector_pasos:
  for semilla in vector_seed:
    for red in vector_red:
      for num_env in vector_numero_env:
        for tipo in vector_tipos_env:
          # Ruta completa para PPO2
          ruta_completa_ppo2 = ruta_base_experimentacion_avanzada + "COMPLETO/"  + str(pasos) + "_timesteps" + "/" + str(semilla) + "_seed" + "/" + "PPO2" + "/" + red + "/" + str(num_env) + "/" + tipo
          logger.configure(dir=ruta_completa_ppo2)
          print(logger.get_dir())
          print(save_inicial)
          model = PPO2_run_agent_train(_, ruta_completa_ppo2, tipo, num_env, red, pasos, semilla, save_inicial)
          model.save(ruta_completa_ppo2 + "/checkpoints/final_save")
          save_inicial = ruta_completa_ppo2 + "/checkpoints/final_save"
          results = pu.load_results(ruta_completa_ppo2)
          grafica = pu.plot_results(results, average_group=True)
          grafica[0].savefig(ruta_completa_ppo2 + "/final_grafica.png")

## Bateria de experimentos cíclicos correspondientes con el conjunto 1 de escenarios de dificultad progresiva (partido)

In [ ]:
#Creación de vectores de parámetros para que sean recorridos y realizar todas las combinaciones posibles en la experimentación
#Vector con diferentes escenarios del entorno
vector_tipos_env = ['academy_single_goal_versus_lazy', '11_vs_11_easy_stochastic', '11_vs_11_stochastic', '11_vs_11_hard_stochastic']
#Vector con numero de multi-entornos a utilizar
vector_numero_env = [8]
#Vector con tipo de red subyacente
vector_red =  ['gfootball_impala_cnn','cnn']
#Vector con numeros de timesteps a utilizar
vector_pasos = [1000000]
#Vector con semillas a utilizar
vector_seed = [0]

In [ ]:
# Ruta para el save inicial desde el que partir en el entrenamiento progresivo

#Ruta base para la experimentacion básica
ruta_base_experimentacion_avanzada= "/content/gdrive/My Drive/TFM/resultados/Experimentacion_Avanzada/"

# Inicializado a None puesto que en el primer entreno no hay save
save_inicial = None
for ciclo in range(3):
  for pasos in vector_pasos:
    for semilla in vector_seed:
      for red in vector_red:
        for num_env in vector_numero_env:
          for tipo in vector_tipos_env:
            # Ruta completa para PPO2
            ruta_completa_ppo2 = ruta_base_experimentacion_avanzada + "PARTIDO-CICLOS/CICLO" + str(ciclo) + "/" + str(pasos) + "_timesteps" + "/" + str(semilla) + "_seed" + "/" + "PPO2" + "/" + red + "/" + str(num_env) + "/" + tipo
            logger.configure(dir=ruta_completa_ppo2)
            print(logger.get_dir())
            print(save_inicial)
            model = PPO2_run_agent_train(_, ruta_completa_ppo2, tipo, num_env, red, pasos, semilla, save_inicial)
            model.save(ruta_completa_ppo2 + "/checkpoints/final_save")
            save_inicial = ruta_completa_ppo2 + "/checkpoints/final_save"
            results = pu.load_results(ruta_completa_ppo2)
            grafica = pu.plot_results(results, average_group=True)
            grafica[0].savefig(ruta_completa_ppo2 + "/final_grafica.png")

## Bateria de experimentos cíclicos correspondientes con el conjunto 2 de escenarios de dificultad progresiva (contragolpe)

In [ ]:
#Creación de vectores de parámetros para que sean recorridos y realizar todas las combinaciones posibles en la experimentación
#Vector con diferentes escenarios del entorno
vector_tipos_env = ['academy_counterattack_easy', 'academy_counterattack_hard']    
#Vector con numero de multi-entornos a utilizar
vector_numero_env = [8]
#Vector con tipo de red subyacente
vector_red =  ['gfootball_impala_cnn','cnn']
#Vector con numeros de timesteps a utilizar
vector_pasos = [1000000]
#Vector con semillas a utilizar
vector_seed = [0]

In [ ]:
# Ruta para el save inicial desde el que partir en el entrenamiento progresivo

#Ruta base para la experimentacion básica
ruta_base_experimentacion_avanzada= "/content/gdrive/My Drive/TFM/resultados/Experimentacion_Avanzada/"

# Inicializado a None puesto que en el primer entreno no hay save
save_inicial = None
for ciclo in range(3):
  for pasos in vector_pasos:
    for semilla in vector_seed:
      for red in vector_red:
        for num_env in vector_numero_env:
          for tipo in vector_tipos_env:
            # Ruta completa para PPO2
            ruta_completa_ppo2 = ruta_base_experimentacion_avanzada + "CONTRAGOLPE-CICLOS/CICLO" + str(ciclo) + "/" + str(pasos) + "_timesteps" + "/" + str(semilla) + "_seed" + "/" + "PPO2" + "/" + red + "/" + str(num_env) + "/" + tipo
            logger.configure(dir=ruta_completa_ppo2)
            print(logger.get_dir())
            print(save_inicial)
            model = PPO2_run_agent_train(_, ruta_completa_ppo2, tipo, num_env, red, pasos, semilla, save_inicial)
            model.save(ruta_completa_ppo2 + "/checkpoints/final_save")
            save_inicial = ruta_completa_ppo2 + "/checkpoints/final_save"
            results = pu.load_results(ruta_completa_ppo2)
            grafica = pu.plot_results(results, average_group=True)
            grafica[0].savefig(ruta_completa_ppo2 + "/final_grafica.png")